In [76]:
# 📌 Install required packages
!pip install SPARQLWrapper osmtogeojson folium

In [77]:
# ✅ Import libraries
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
import folium
from folium import LayerControl
from osmtogeojson import osmtogeojson
import json


In [78]:
# ✅ 1. Query Wikidata for SAT-connected nature reserves with OSM relation/way
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setReturnFormat(JSON)
sparql.setQuery("""
SELECT DISTINCT ?item ?itemLabel ?SATLabel ?P402 ?P10689 WHERE {
  wd:Q131318799 wdt:P3018 ?item.
  OPTIONAL { ?item wdt:P402 ?P402. }
  OPTIONAL { ?item wdt:P10689 ?P10689. }
  OPTIONAL {
    ?item wdt:P2789 ?SATid.
    ?SATid wdt:P361 wd:Q131318799.
    ?SATid rdfs:label ?SATLabel. FILTER(LANG(?SATLabel)="sv")
  }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
}
""")
results = sparql.query().convert()

In [79]:
#results

In [80]:
# ✅ 2. Build list of features with OSM IDs
items = []
for r in results["results"]["bindings"]:
    def val(k): return r[k]["value"] if k in r else None
    label = val("itemLabel")
    sat = val("SATLabel")
    rel = val("P402")
    way = val("P10689")

    if rel:
        overpass = f"relation({rel});"
    elif way:
        overpass = f"way({way});"
    else:
        continue

    items.append({
        "label": label,
        "sat": sat or "Okänt avsnitt",
        "overpass": overpass
    })

print(f"Loaded {len(items)} reserves with OSM geometry.")

Loaded 13 reserves with OSM geometry.


In [81]:
items

[{'label': 'Fjärdlångs naturreservat',
  'sat': 'SAT Fjärdlång',
  'overpass': 'relation(1463788);'},
 {'label': 'Nämdö naturreservat',
  'sat': 'SAT Nämdö',
  'overpass': 'relation(1463777);'},
 {'label': 'Utö naturreservat',
  'sat': 'SAT Utö',
  'overpass': 'relation(1463748);'},
 {'label': 'Grinda naturreservat',
  'sat': 'SAT Grinda',
  'overpass': 'relation(1463789);'},
 {'label': 'Lidö naturreservat',
  'sat': 'SAT Lidö',
  'overpass': 'relation(1463761);'},
 {'label': 'Norra skogens naturreservat',
  'sat': 'SAT Ornö',
  'overpass': 'way(1331536835);'},
 {'label': 'Arholma-Idö naturreservat',
  'sat': 'SAT Arholma',
  'overpass': 'relation(1463750);'},
 {'label': 'Ålö-Rånö naturreservat',
  'sat': 'SAT Ålö',
  'overpass': 'relation(1463767);'},
 {'label': 'Ålö-Rånö naturreservat',
  'sat': 'SAT Rånö',
  'overpass': 'relation(1463767);'},
 {'label': 'Finnhamns naturreservat',
  'sat': 'SAT Finnhamn',
  'overpass': 'relation(1463800);'},
 {'label': 'Öja-Landsort Naturvårdsområde'

In [82]:
# ✅ 3. Fetch geometry from Overpass
query_body = "[out:json][timeout:25];(" + "".join(item["overpass"] for item in items) + ");(._;>;);out body;>;out skel qt;"
overpass_url = "https://overpass-api.de/api/interpreter"
response = requests.post(overpass_url, data={"data": query_body})
osm_data = response.json()

In [83]:
query = """
[out:json][timeout:25];
relation(1463748);
(._;>;);
out body;
"""


In [85]:
#osm_data

In [86]:
ways = [el for el in osm_data['elements'] if el['type'] == 'way']
relations = [el for el in osm_data['elements'] if el['type'] == 'relation']
print(f"Ways: {len(ways)}, Relations: {len(relations)}")

Ways: 274, Relations: 10


In [87]:
missing_nodes = False
node_ids = {n["id"] for n in osm_data["elements"] if n["type"] == "node"}
for way in ways:
    for nid in way.get("nodes", []):
        if nid not in node_ids:
            print(f"Missing node {nid} for way {way['id']}")
            missing_nodes = True
if not missing_nodes:
    print("✅ All nodes required for ways are present.")

✅ All nodes required for ways are present.


In [88]:
geojson = json2geojson(osm_data)


In [90]:
import json
#print(json.dumps(geojson, indent=2))


In [91]:
ways_with_tags = [w for w in ways if 'tags' in w and w['tags']]
print(f"Ways with tags: {len(ways_with_tags)}")

Ways with tags: 41


In [92]:
relations_with_tags = [r for r in relations if 'tags' in r and r['tags']]
print(f"Relations with tags: {len(relations_with_tags)}")


Relations with tags: 10


In [93]:
from osm2geojson import json2geojson

try:
    geojson = json2geojson(osm_data)
    features = geojson['features']
    print(f"✅ Converted {len(features)} features to GeoJSON.")
    for i, feat in enumerate(features[:5]):  # show first 5
        print(f"- Feature {i}: {feat['properties'].get('name')}")
except Exception as e:
    print("❌ Failed to convert OSM to GeoJSON:", e)

✅ Converted 12 features to GeoJSON.
- Feature 0: None
- Feature 1: None
- Feature 2: None
- Feature 3: None
- Feature 4: None


In [94]:
import geojson

nodes_by_id = {el["id"]: el for el in osm_data["elements"] if el["type"] == "node"}
features = []

for way in ways_with_tags:
    coords = []
    for nid in way.get("nodes", []):
        node = nodes_by_id.get(nid)
        if node:
            coords.append((node["lon"], node["lat"]))
    if coords and coords[0] == coords[-1] and len(coords) >= 4:
        features.append(geojson.Feature(
            geometry=geojson.Polygon([coords]),
            properties=way["tags"]
        ))

print(f"✅ Manually created {len(features)} features.")
geojson_data = geojson.FeatureCollection(features)


✅ Manually created 27 features.


In [95]:
import folium

m = folium.Map(location=[59.3, 18.5], zoom_start=9)

folium.GeoJson(
    geojson_data,
    name="Nature Reserves",
    tooltip=folium.GeoJsonTooltip(fields=["name"]),
    style_function=lambda feature: {
        'fillColor': '#228B22',
        'color': '#006400',
        'weight': 2,
        'fillOpacity': 0.5,
    }
).add_to(m)

folium.LayerControl().add_to(m)
m

In [96]:
import json

with open("nature_reserves.geojson", "w") as f:
    json.dump(geojson_data, f)

In [97]:
import geojson

# Index all elements
nodes_by_id = {el["id"]: el for el in osm_data["elements"] if el["type"] == "node"}
ways_by_id = {el["id"]: el for el in osm_data["elements"] if el["type"] == "way"}
relations = [el for el in osm_data["elements"] if el["type"] == "relation"]
ways = [el for el in osm_data["elements"] if el["type"] == "way"]

features = []

# --- Convert tagged, closed ways to polygons ---
for way in ways:
    if "tags" not in way:
        continue
    node_coords = []
    for nid in way.get("nodes", []):
        node = nodes_by_id.get(nid)
        if node:
            node_coords.append((node["lon"], node["lat"]))
    if len(node_coords) >= 4 and node_coords[0] == node_coords[-1]:
        features.append(geojson.Feature(
            geometry=geojson.Polygon([node_coords]),
            properties=way["tags"]
        ))

# --- Convert relations with multipolygon or boundary type ---
for rel in relations:
    if "tags" not in rel or rel.get("tags", {}).get("type") not in {"multipolygon", "boundary"}:
        continue

    outer_rings = []
    inner_rings = []

    for member in rel.get("members", []):
        if member["type"] != "way":
            continue
        way = ways_by_id.get(member["ref"])
        if not way:
            continue
        coords = []
        for nid in way.get("nodes", []):
            node = nodes_by_id.get(nid)
            if node:
                coords.append((node["lon"], node["lat"]))
        if len(coords) >= 4 and coords[0] == coords[-1]:
            if member.get("role") == "outer":
                outer_rings.append(coords)
            elif member.get("role") == "inner":
                inner_rings.append(coords)

    if outer_rings:
        features.append(geojson.Feature(
            geometry=geojson.Polygon([*outer_rings, *inner_rings]) if len(outer_rings) == 1
            else geojson.MultiPolygon([([*outer_rings[0], *inner_rings])]),
            properties=rel["tags"]
        ))

# Wrap as GeoJSON FeatureCollection
geojson_data = geojson.FeatureCollection(features)
print(f"✅ Created {len(features)} features (ways + relations).")


✅ Created 30 features (ways + relations).


In [98]:
import folium

m = folium.Map(location=[59.2, 18.5], zoom_start=9)

folium.GeoJson(
    geojson_data,
    name="Nature Reserves",
    style_function=lambda feature: {
        'fillColor': '#228B22',
        'color': '#006400',
        'weight': 2,
        'fillOpacity': 0.4,
    },
    tooltip=folium.GeoJsonTooltip(fields=["name"], aliases=["Reservat"])
).add_to(m)

folium.LayerControl().add_to(m)
m
